(cartopy)=

# Cartopy

An executive summary ... its functions ... authors ... and history ...

Cartopy is a python library that can be used in combination with matplotlib to create geographical maps. 

Starting in 2016, The Cartopy Project began development as a replacement for Basemap.

Cartopy has 28 people listed as contributors.

Pros of Cartopy:
- Relatively easy to learn
- Many different types of mapping projections
- Great options to color in the map

Cons of Cartopy:
- Slow loading times for complex maps
- Alternate files often required for high precision mapping

Sources: 
- https://matplotlib.org/basemap/
- https://scitools.org.uk/cartopy/docs/latest/


Use the NYC MV Collisons-Crashes data to demonstrate ...

Think of questions of interest that can be answered by visualization
...